# Image Classification

This notebook shows how to implement image classification in tensorflow. It's based on images of natural scenes around the world. 
It will allow you to classify buildings, forests, glaciers, mountains, sea and street scenery.

As for the machine learning, the following concepts are covered:

- Data generators:
    - image scaling
    - labels encoding
    - batch loading
- Data augmentation
- Convolutional Neural Networks
    - convolutional layers
    - pooling layers
    - fully-connected layers
    - dropout
- Callbacks
- Transfer learning


## Data

The notebook is based on Intel Image Classification dataset. It contains 25k images of various landscapes. After building (and training) the neural network, you will be able to classify any landscape image as belonging to one of those categories:
- buildings
- forest
- glacier
- mountain
- sea
- street

To start, [download the data](https://www.kaggle.com/puneet6060/intel-image-classification/download) and place it in your working directory. 





In [ ]:
import os
import random

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from utils import (AccuracyCallback, pick_files_from_directory, plot_training_progress,
                   plot_sample_images, unpack_file, visualize_convolutions)

In [ ]:
# Configure GPU
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: 
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
DATA_FILE = "intel-image-classification.zip"
DATA_DIR = "data/images"

unpack_file(DATA_FILE, DATA_DIR)

TRAIN_DIR = DATA_DIR + "/seg_train/seg_train"
TEST_DIR = DATA_DIR + "/seg_test/seg_test"

In [ ]:
# Let's plot sample images

categories = os.listdir(TRAIN_DIR)
print("Categories are: {}".format(', '.join(categories)))

%matplotlib inline
plot_sample_images(TRAIN_DIR, categories, 4)

In [ ]:
# Process input data. Include data augmentation.
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  rotation_range=40,
                                                                  width_shift_range=0.2,
                                                                  height_shift_range=0.2,
                                                                  shear_range=0.2,
                                                                  zoom_range=0.2,
                                                                  horizontal_flip=True,
                                                                  fill_mode='nearest')

train_generator = image_generator.flow_from_directory(
    TRAIN_DIR, target_size=(150, 150), class_mode="categorical", batch_size=32)

test_generator = image_generator.flow_from_directory(
    TEST_DIR, target_size=(150, 150), class_mode="categorical", batch_size=32)

## Convolutional Neural Network

Let's build a simple conv net for multiclass classification.

In [ ]:
# Model architecture

vanilla_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation="relu", input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(6, activation="softmax")
])

print(vanilla_model.summary())

In [ ]:
# Now it's time to train our model. 

tf.keras.backend.clear_session()

accuracy_callback = AccuracyCallback(0.9)

vanilla_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
                      loss="categorical_crossentropy",
                      metrics=["acc"])

history_vanilla = vanilla_model.fit(train_generator,
                                    validation_data=test_generator,
                                    epochs=2,
                                    verbose=1,
                                    callbacks=[accuracy_callback])

In [ ]:
# Let's evaluate its performance on test data

test_metrics = vanilla_model.evaluate(test_generator)

for i in range(len(test_metrics)):
    print("Test {}: {}".format(vanilla_model.metrics_names[i], test_metrics[i]))
    
plot_training_progress(history_vanilla, "acc")
plot_training_progress(history_vanilla, "loss")

We have evaluated models by accuracy and log loss. Now, let's see how our model performs on random pictures. 

Can it correctly regognize the landscape images of your choice?

In [ ]:
# Define list of images you would like to classify (full path).

files = []
for category in categories:
    files += pick_files_from_directory(os.path.join(TEST_DIR, category), k=2)

In [ ]:
# Let's classify those images 

inputs = []
for file in files: 
    img = tf.keras.preprocessing.image.load_img(file, target_size=(150, 150, 3))
    x = tf.keras.preprocessing.image.img_to_array(img)
    inputs.append(np.expand_dims(x, axis=0))

images = np.vstack(inputs)
labels = vanilla_model.predict_classes(images)

id_to_class = {value: key for key, value in train_generator.class_indices.items()} 

In [ ]:
# Plottng results. How may did you get wrong?
fig = plt.gcf()
fig.set_size_inches(15, 50)

i = 0
for file, label in zip(files, labels):
    
    sp = plt.subplot(int(len(files)/2), 2, i+1)
    sp.axis("Off")
    
    img = mpimg.imread(file)
    plt.imshow(img)
    
    plt.title("Predicted: {}".format(id_to_class[label]))
    i+=1
    
plt.show()

## Transfer Learning

Can we do any better than that? 

Instead of training our model for long hours, let us build upon well-trained models. 
We will use InceptionV3 model traned on ImageNet. We will modify a couple of last layers to allow the network to learn features specific to out landscape dataset. 

In [ ]:
# Now, let's implement transfer learning

pretrained_model = tf.keras.applications.InceptionV3(input_shape=(150, 150, 3),
                                                      include_top=False,
                                                      weights='imagenet')

for layer in pretrained_model.layers:
    layer.trainable = False


last_layer = pretrained_model.get_layer("mixed8")
last_output = last_layer.output

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(6, activation="softmax")(x)

tl_model = tf.keras.models.Model(pretrained_model.input, x)

tl_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
                 loss="categorical_crossentropy",
                 metrics=["acc"])

print(tl_model.summary())

In [ ]:
# Time to train model based on transfer learning. 
tf.keras.backend.clear_session()
            
tl_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), 
                 loss="categorical_crossentropy", 
                 metrics=["acc"])

history_tl = tl_model.fit(train_generator, 
                          validation_data=test_generator, 
                          epochs=1, 
                          verbose=1, 
                          callbacks=[accuracy_callback])

In [ ]:
# Let's evaluate its performance on test data. Does it do any better than the model we built from scratch?

test_metrics = tl_model.evaluate(test_generator)

for i in range(len(test_metrics)):
    print("Test {}: {}".format(vanilla_model.metrics_names[i], test_metrics[i]))
    
plot_training_progress(history_tl, "acc")
plot_training_progress(history_tl, "loss")

## Visualizing convolutions

We have seen how to build and train image classification model. The core part of this model, are convolutions and pooling layers. Let's have a look at what do they actually do. 

Who does an image look like after passing through each Conv2D and MaxPooling layer?

In [ ]:
# Visualize convolutions
category = "forest"
model = vanilla_model
img = pick_files_from_directory(os.path.join(TRAIN_DIR, category))[0]

visualize_convolutions(model, img)